# Predictions


## Data selection

Here, we will define a dataset with no labels in aim to make prediction on the embedding columns by using our trained model.

In [13]:
'''
Here is the data importation and all functions that we use to create our labels
'''

## Data Importation

import pandas as pd
import io
from ast import literal_eval
import re
import numpy as np

def converter(x):
    return literal_eval(x)
def convert_to_array(x):
    x = x.replace('\n','')[1:-1]
    return np.array(list(map(float,x.split())))

#define each column
converters={'Encode Text': converter, 'embedding':convert_to_array}

df_with_label = pd.read_csv('C:/Users/david/Downloads/df_with_label.csv', 
                 sep=',',index_col=0,
                 converters=converters)
df_no_label = pd.read_csv('C:/Users/david/Downloads/df_no_label.csv', 
                 sep=',',index_col=0,
                 converters=converters)
data = df_with_label.copy()


## Labelling

### Cleaning function

def clean_years(string):
    string.replace('3-5','4',inplace=True)
    string.replace('(5','5',inplace=True)
    string.replace('5-7','6',inplace=True)
    string.replace('7-10','8',inplace=True)
    string.replace('2-3','3',inplace=True)
    string.replace('(3','3',inplace=True)
    string.replace('(2','2',inplace=True)
    string.replace('(10','10',inplace=True)
    string.replace('1-3','2',inplace=True)
    string.replace('2-5','4',inplace=True)
    string.replace('8-10','9',inplace=True)
    string.replace('5-10','8',inplace=True)
    string.replace('Five','5',inplace=True)
    string.replace('1-2','2',inplace=True)

    listOfYears =  ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20']
    
    string_cleaned = string.apply(lambda x : None if x not in listOfYears else int(x) )
    return string_cleaned

### Extract function


def extract_years(description):
    
    # extract the years of experience on the description columns
        
    years = description.apply(lambda x : x.split("+ years")[0].split()[-1] )
    
    # Clean the extracted years:
    years = clean_years(years)
    return years

def extract_adj(title):
    
    Adj = {'entry', 'i', 'intern', 'junior', 'intership',
           'associate', 'ii', 'staff', 'mid','mid-senior',
           'sr','sr.','senior', 'lead','manager','iii', 'director', 'executive', 'president', 'vice'}
    
    for word in title.lower().split():
        if word in Adj: 
            return word
    return None
    
    


### Labelling function 

def Labelled(df):
    '''
    This function label jobs base of years of experience and key word in the type

    We have two labels Junior and Senior

    A job will be a Junior level if it has Junior keys word in job title, If it has mid senior key in your job title and less than 4 years or it don't have a key word in job title and have less than 4 years of experience

    A job will be a Senior level if it has Senior keys word in job title, If it has mid senior key in your job title and more than 4 years or it don't have a key word in job title and have more than 4 years of experience


    '''

    junior_keys = {'entry', 'i', 'intern', 'junior', 'intership'}
    mid_keys = {'associate', 'ii', 'staff', 'mid', 'mid-senior'}
    senior_keys = {'sr','sr.','senior', 'lead','manager','iii', 'director', 'executive', 'president', 'vice'}
    
    for word in df['Title'].lower().split():
        # If a job has a junior keyword then it is a junior job
        if word in junior_keys:
            
            return "Junior"
            
        # If a job has a junior keyword and years of experience is less than 4 then it is a junior job
        elif word in mid_keys:
            
            if df['years of experience'] < 4 :
                return "Junior"
            elif df['years of experience'] > 4:
                
                return "Senior"

        # If job title has senior keyword than it is a senior job
        elif word in senior_keys:

            return "Senior"

        # when we don't have any key word in the title
        # less than 4 years of experience the job is a junior job and more is Senior
        else:
            if df['years of experience'] < 4 :
                return "Junior"
            elif df['years of experience'] > 4:
                return "Senior"

### Apply Functions

In [14]:
data['years of experience'] = extract_years(data['Description'])

data['key title'] = data['Title'].apply(extract_adj)

data['new_label'] = data.apply(Labelled, axis= 1)


In [16]:
data

,Title,Company,Location,Link,Description,seniority_level,query,job_id,job_index,apply_link,...,employment_type,description_html,place,New Description,Encode Text,New_Description_1,embedding,years of experience,key title,new_label
2108,Data Scientist,CBRE,Ontario,https://ca.linkedin.com/jobs/view/data-scienti...,Data Scientist\n\nJob ID\n\n26195\n\nPosted\n\...,Entry level,AI Engineer,2.673220e+09,0.0,https://www.linkedin.com/jobs/view/externalApp...,...,Full-time,"<div class=""description__text description__tex...","Toronto, Ontario, Canada",Data Scientist. Job ID. 26195. Posted. 03-...,[Develop and deploy cloud-based automation app...,Data Scientist. Job ID. 26195. Posted. 03-...,"[-0.29719907, -0.19061026, 0.818259895, -0.405...",2.0,None,Junior
2109,Machine Learning Engineer - DeviantArt,Wix.com,British Columbia,https://ca.linkedin.com/jobs/view/machine-lear...,We Are\n\nThe DeviantArt team. We’ve been arou...,Mid-Senior level,AI Engineer,2.673214e+09,0.0,https://www.linkedin.com/jobs/view/externalApp...,...,Full-time,"<div class=""description__text description__tex...","Vancouver, British Columbia, Canada",We Are. The DeviantArt team. We’ve been aroun...,[The systems we develop are at the core of how...,We Are. The DeviantArt team. We’ve been aroun...,"[-0.0684582368, -0.242557198, 0.729946375, -0....",2.0,None,Junior
2110,Data Scientist,Scribd,British Columbia,https://ca.linkedin.com/jobs/view/data-scienti...,"At Scribd (pronounced “scribbed”), we believe ...",Associate,AI Engineer,2.635379e+09,1.0,https://www.linkedin.com/jobs/view/externalApp...,...,Full-time,"<div class=""description__text description__tex...","Vancouver, British Columbia, Canada","At Scribd (pronounced “scribbed”), we believe ...",[Applied Research works on a variety of ML and...,"At Scribd (pronounced “scribbed”), we believe ...","[0.0843317956, -0.296700656, 0.893622339, -0.5...",3.0,None,Junior
2112,Machine Learning Developer / Développeur appre...,Unity Technologies,Quebec,https://ca.linkedin.com/jobs/view/machine-lear...,The opportunity\n\nUnity is actively seeking a...,Entry level,AI Engineer,2.667106e+09,1.0,https://www.linkedin.com/jobs/view/externalApp...,...,Full-time,"<div class=""description__text description__tex...","Montreal, Quebec, Canada",The opportunity. Unity is actively seeking a ...,[Unity is actively seeking a proficient machin...,The opportunity. Unity is actively seeking a ...,"[-0.0671256483, -0.0651571602, 0.861628294, -0...",NaN,None,None
2113,GPU Machine Learning Software Developer- 104421,AMD,Alberta,https://ca.linkedin.com/jobs/view/gpu-machine-...,What You Do At AMD Changes Everything\n\nAt AM...,Mid-Senior level,AI Engineer,2.635379e+09,1.0,NaN,...,Full-time,"<div class=""description__text description__tex...","Calgary, Alberta, Canada",What You Do At AMD Changes Everything. At AMD...,[We believe in changing the world for the bett...,What You Do At AMD Changes Everything. At AMD...,"[0.0655079782, 0.0121015627, 0.840936542, -0.5...",NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47699,"Senior Manager, Software Engineering - Open So...",GitHub,Ontario,https://ca.linkedin.com/jobs/view/senior-manag...,\nGitHub is changing the way the world builds ...,Mid-Senior level,Software Developer,2.674553e+09,106.0,NaN,...,Full-time,"<div class=""description__text description__tex...","Toronto, Ontario, Canada",. GitHub is changing the way the world builds ...,"[The team is distributed across the US, Canada...",. GitHub is changing the way the world builds ...,"[-0.0730892941, -0.308131516, 0.797860384, -0....",NaN,senior,Senior
47701,"Senior Software Developer - .NET, Upland (Remo...",Crossover for Work,Ontario,https://ca.linkedin.com/jobs/view/senior-softw...,Crossover is the world's #1 source of remote f...,Associate,Software Developer,2.673898e+09,111.0,NaN,...,Full-time,"<div class=""description__text description__tex...","Toronto, Ontario, Canada",Crossover is the world's

### Select rows with no labels

In [22]:
data_unlabelled = data[(data['new_label'] != 'Junior') & (data['new_label'] != 'Senior' )]

data_unlabelled = data_unlabelled.reset_index()[['Title','Description','embedding','years of experience','key title','new_label']]

data_unlabelled

,Title,Description,embedding,years of experience,key title,new_label
0,Machine Learning Developer / Développeur appre...,The opportunity\n\nUnity is actively seeking a...,"[-0.0671256483, -0.0651571602, 0.861628294, -0...",NaN,None,None
1,GPU Machine Learning Software Developer- 104421,What You Do At AMD Changes Everything\n\nAt AM...,"[0.0655079782, 0.0121015627, 0.840936542, -0.5...",NaN,None,None
2,Machine Learning Engineer,Company Description\n\nWish is a mobile e-comm...,"[-0.0975675657, -0.31235218, 0.956554711, -0.4...",NaN,None,None
3,Back End Developer,About the Job:\n\nThe Sama R&D team is focused...,"[-0.0985258818, -0.330438614, 0.894662619, -0....",NaN,None,None
4,"Machine Learning Engineer in Vancouver, BC, Ca...",We are looking for a Machine Learning Engineer...,"[-0.06765608, 0.11911686, 0.3978404, -0.348407...",NaN,None,None
...,...,...,...,...,...,...
14927,Software Developer- Big data,Ready to make an impact? Arctic Wolf seeks a S...,"[0.0725050345, -0.115438573, 0.61423105, -0.94...",NaN,None,None
14928,Technology Specialist,"Who are we?\n\nAt Amdocs, our purpose is to en...","[0.04446964, -0.34980673, 0.87086874, -0.66928...",NaN,None,None
14929,Developer II,Les candidats référés ne doivent pas postuler ...,"[-0.367371976, 0.062841855, 0.184929281, -0.89...",NaN,ii,None
14930,Frontend Developer,"Hi,\n\n\n\n\nWe are Iris Software, one of our ...","[-0.323665112, -0.424904585, 0.661687434, -0.8...",NaN,None,None


## Predictions

For this prediction, we will use rows which has no label that means rows which have no years of experience and key title

Since the embedding column is a pandas series which contain a list, we always need to transform it in a numpy array before before sending it to the model

In [29]:

from joblib import Parallel, delayed
import joblib
  
  
# Load the model from the file
saved_model = joblib.load('C:/Users/david/Desktop/SeniorClassificationModel.pkl')


X_origi = data_unlabelled[['embedding']]

# Transform a coulumn of list in array 
X = np.array([list(X_origi.iloc[i,0]) for i in range(len(X_origi))])


# Use the loaded model to make predictions
saved_model.predict(X)

array([1, 1, 1, ..., 1, 1, 1])

In [27]:
predictions = pd.Series(saved_model.predict(X)).apply( lambda x: 'Junior' if x == 0 else 'Senior')

data_unlabelled['predictions'] = predictions

data_unlabelled

,Title,Description,embedding,years of experience,key title,new_label,predictions
0,Machine Learning Developer / Développeur appre...,The opportunity\n\nUnity is actively seeking a...,"[-0.0671256483, -0.0651571602, 0.861628294, -0...",NaN,None,None,Senior
1,GPU Machine Learning Software Developer- 104421,What You Do At AMD Changes Everything\n\nAt AM...,"[0.0655079782, 0.0121015627, 0.840936542, -0.5...",NaN,None,None,Senior
2,Machine Learning Engineer,Company Description\n\nWish is a mobile e-comm...,"[-0.0975675657, -0.31235218, 0.956554711, -0.4...",NaN,None,None,Senior
3,Back End Developer,About the Job:\n\nThe Sama R&D team is focused...,"[-0.0985258818, -0.330438614, 0.894662619, -0....",NaN,None,None,Junior
4,"Machine Learning Engineer in Vancouver, BC, Ca...",We are looking for a Machine Learning Engineer...,"[-0.06765608, 0.11911686, 0.3978404, -0.348407...",NaN,None,None,Senior
...,...,...,...,...,...,...,...
14927,Software Developer- Big data,Ready to make an impact? Arctic Wolf seeks a S...,"[0.0725050345, -0.115438573, 0.61423105, -0.94...",NaN,None,None,Senior
14928,Technology Specialist,"Who are we?\n\nAt Amdocs, our purpose is to en...","[0.04446964, -0.34980673, 0.87086874, -0.66928...",NaN,None,None,Senior
14929,Developer II,Les candidats référés ne doivent pas postuler ...,"[-0.367371976, 0.062841855, 0.184929281, -0.89...",NaN,ii,None,Senior
14930,Frontend Developer,"Hi,\n\n\n\n\nWe are Iris Software, one of our ...","[-0.323665112, -0.424904585, 0.661687434, -0.8...",NaN,None,None,Senior


## Finale Table with predictions

In [30]:
data_unlabelled

,Title,Description,embedding,years of experience,key title,new_label,predictions
0,Machine Learning Developer / Développeur appre...,The opportunity\n\nUnity is actively seeking a...,"[-0.0671256483, -0.0651571602, 0.861628294, -0...",NaN,None,None,Senior
1,GPU Machine Learning Software Developer- 104421,What You Do At AMD Changes Everything\n\nAt AM...,"[0.0655079782, 0.0121015627, 0.840936542, -0.5...",NaN,None,None,Senior
2,Machine Learning Engineer,Company Description\n\nWish is a mobile e-comm...,"[-0.0975675657, -0.31235218, 0.956554711, -0.4...",NaN,None,None,Senior
3,Back End Developer,About the Job:\n\nThe Sama R&D team is focused...,"[-0.0985258818, -0.330438614, 0.894662619, -0....",NaN,None,None,Junior
4,"Machine Learning Engineer in Vancouver, BC, Ca...",We are looking for a Machine Learning Engineer...,"[-0.06765608, 0.11911686, 0.3978404, -0.348407...",NaN,None,None,Senior
...,...,...,...,...,...,...,...
14927,Software Developer- Big data,Ready to make an impact? Arctic Wolf seeks a S...,"[0.0725050345, -0.115438573, 0.61423105, -0.94...",NaN,None,None,Senior
14928,Technology Specialist,"Who are we?\n\nAt Amdocs, our purpose is to en...","[0.04446964, -0.34980673, 0.87086874, -0.66928...",NaN,None,None,Senior
14929,Developer II,Les candidats référés ne doivent pas postuler ...,"[-0.367371976, 0.062841855, 0.184929281, -0.89...",NaN,ii,None,Senior
14930,Frontend Developer,"Hi,\n\n\n\n\nWe are Iris Software, one of our ...","[-0.323665112, -0.424904585, 0.661687434, -0.8...",NaN,None,None,Senior
